In [1]:
# 创建数据集
from mxnet import ndarray as nd
from mxnet import autograd
from mxnet import gluon

num_inputs = 2
num_examples = 1000

true_w = [2, -3.4]
true_b = 4.2

X = nd.random_normal(shape=(num_examples, num_inputs))
y = true_w[0] * X[:, 0] + true_w[1] * X[:, 1] + true_b
y += 0.01 * nd.random_normal(shape=y.shape)

In [2]:
# 数据读取

batch_size = 10
dataset = gluon.data.ArrayDataset(X, y)
data_iter = gluon.data.DataLoader(dataset, batch_size, shuffle=True)

In [3]:
for data, label in data_iter:
    print(data, label)
    break


[[-1.0981429e+00  6.0562080e-01]
 [ 2.3105950e+00  1.1056191e+00]
 [-9.4586486e-01  5.6785142e-01]
 [ 5.4093814e-01  5.2064251e-02]
 [ 1.6388480e-01  3.0105186e-01]
 [ 1.2054669e+00 -9.9645287e-01]
 [-1.3334060e+00  1.1095357e+00]
 [-6.6284835e-04 -1.4287020e+00]
 [ 1.7928383e+00 -9.9927567e-02]
 [-1.4626571e+00 -4.7491392e-01]]
<NDArray 10x2 @cpu(0)> 
[-0.05260437  5.0523906   0.39167407  5.1196246   3.5110636   9.992634
 -2.2257197   9.052385    8.135473    2.882936  ]
<NDArray 10 @cpu(0)>


In [4]:
# 定义模型
net = gluon.nn.Sequential()

In [5]:
net.add(gluon.nn.Dense(1))

In [6]:
# 初始化模型参数
net.initialize()

In [7]:
# 损失函数
square_loss = gluon.loss.L2Loss()

In [8]:
# 优化
trainer = gluon.Trainer(
    net.collect_params(), 'sgd', {'learning_rate': 0.1})

In [9]:
# 训练
epochs = 5
for e in range(epochs):
    total_loss = 0
    for data, label in data_iter:
        with autograd.record():
            output = net(data)
            loss = square_loss(output, label)
        loss.backward()
        trainer.step(batch_size)
        total_loss += nd.sum(loss).asscalar()
    print("Epoch %d, average loss: %f" % (e, total_loss/num_examples))

Epoch 0, average loss: 0.908515
Epoch 1, average loss: 0.000051
Epoch 2, average loss: 0.000051
Epoch 3, average loss: 0.000051
Epoch 4, average loss: 0.000051


In [14]:
dense = net[0]
true_w, dense.weight.data()

([2, -3.4], 
 [[ 2.00005   -3.3991935]]
 <NDArray 1x2 @cpu(0)>)

In [17]:
true_b, dense.bias.data()

(4.2, 
 [4.200748]
 <NDArray 1 @cpu(0)>)